### Pytorch에서의 실험 재현 (reproducible)을 위해 고려해야할 randomness 제어

### 1. Pytorch Randomness
- 우선 나는 pytorch 1.7.1을 사용하므로 해당 문서를 참고함 (https://pytorch.org/docs/1.7.1/notes/randomness.html?highlight=randomness)

- `torch.manual_seed(seed)` : to seed the RNG (Random Number Generator) for all devices (both CPU and CUDA)

- 원래는 `torch.backends.cudnn.determinisitic = True` 또는 `torch.backends.cudnn.benchmark = False` 이런식으로 제어.

- torch 1.7 version 부터는 `torch.set_deterministic(bool)` 이 만들어짐

In [1]:
import torch
print(torch.__version__)

1.7.1


In [2]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
torch.manual_seed(0)
torch.set_deterministic(True)

[W Context.cpp:69] Warning: torch.set_deterministic is in beta, and its design and  functionality may change in the future. (function operator())


### 2. CuDNN
- CUDA library인데 cudnn 연산을 deterministic 하게 하면 nondeterministic모다 연산 처리의 속도가 감소됨

- 애초에 nondeterministic 한 operation 들은 (ex. `torch.Tensor.index_add_()`, `torch.Tensor.scatter_add_()`, ...) error message를 내보냄

### 3. Numpy
- numpy가 c로 되어있어서 굉장히 빠른 수학 연산을 제공하는 경우가 많아서 자주 쓰임

- `np.random.seed(seed)` 로 numpy operator를 deterministic하게 만들 수 있음

In [5]:
import numpy as np
np.random.seed(0)

### 4. torchvision.transforms
- torchvision의 transform에는 `RandomCrop()`, `RandomHorizontalFlip()` 등과 같은 randomness가 존재하는 함수가 있음

- 이러한 operator는 torch, numpy, cudnn이 아닌 python의 내부 random 라이브러리에 의해 결정됨

- `random.seed(seed)` 로 파이썬 randomness 제어

In [6]:
import random
random.seed(0)

### 5. Others
- 이것 외에도 torch.cuda 에서 RNG를 제어해주는 방법은 많음 (참고 : https://pytorch.org/docs/1.7.1/cuda.html?highlight=torch%20cuda%20manual#torch.cuda.manual_seed_all)

- 대표적으로는 `torch.cuda.manual_seed(seed)` (current GPU) 또는 `torch.cuda.manual_seed_all(seed)` (all GPUs)를 많이 사용한다고 함